# BM25 Search

## BM25 Formula

The BM25 relevance score for a document $d$ and a query $q$ is computed as:

$$
\text{BM25}(d, q) = \sum_{t \in q} \text{IDF}(t) \cdot \frac{f_{t, d} \cdot (k_1 + 1)}{f_{t, d} + k_1 \cdot (1 - b + b \cdot \frac{|d|}{\text{avgdl}})}
$$

### Where:
1. **$t$**: A term in the query.
2. **$f_{t, d}$**: The frequency of term $t$ in document $d$ (term frequency).
3. **$|d|$**: The length of document $d$ (number of terms in $d$).
4. **$\text{avgdl}$**: The average document length in the corpus.
5. **$\text{IDF}(t)$**: The inverse document frequency of term $t$, calculated as:
   $$
   \text{IDF}(t) = \log\left(\frac{N - n_t + 0.5}{n_t + 0.5} + 1\right)
   $$
   - $N$: Total number of documents.
   - $n_t$: Number of documents containing term $t$.
6. **$k_1$**: Controls the term frequency saturation (commonly set to 1.2).
7. **$b$**: Controls the document length normalization (commonly set to 0.75).


[Explanation](https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables)

## Differences between BM25 and TF-IDF IDF formula:

The IDF formula in BM25 is different from the one used in TF-IDF because BM25 is specifically designed for ranking documents in a search setting. Its IDF calculation incorporates additional considerations to address some of the limitations in the standard TF-IDF formula. Here's a breakdown of the differences and their significance:

---

### TF-IDF IDF Formula
$$
\text{IDF}(t) = \log\left(\frac{N}{n_t}\right)
$$

- **$N$:** Total number of documents in the corpus.
- **$n_t$:** Number of documents containing term $t$.

This formula assigns a higher weight to rare terms (low $n_t$) and a lower weight to frequent terms (high $n_t$). However, it has limitations:
1. **Division by Zero Risk:** If $n_t = 0$, the formula becomes undefined.
2. **Unbounded Values:** The result can become arbitrarily large for very rare terms.
3. **No Smoothing:** Doesn't account for cases where terms are extremely rare but still have some presence.

---

### BM25 IDF Formula
$$
\text{IDF}(t) = \log\left(\frac{N - n_t + 0.5}{n_t + 0.5} + 1\right)
$$

- Adds **0.5** to both numerator and denominator to:
  - Smooth values and prevent division by zero.
  - Ensure terms with very low $n_t$ don’t dominate excessively.
  
- Includes $N - n_t + 0.5$, which considers the number of documents that *do not* contain $t$. This adjustment:
  - Gives a more balanced score when $t$ appears in many documents.
  - Reduces the impact of terms that are either very common or very rare.

---

### Why BM25's IDF is Better for Search
1. **Avoids Extreme Scores:**  
   BM25's IDF formula avoids overly penalizing terms that appear in many documents while ensuring rare terms still get higher weights.
   
2. **Smoothing for Robustness:**  
   The addition of $0.5$ ensures robustness in small datasets where $n_t$ can be 0 or very small.

3. **Balances Common vs. Rare Terms:**  
   - Standard TF-IDF can overemphasize rare terms.
   - BM25 dampens this effect, making it better suited for search where a mix of common and rare terms might be relevant.

---

### Practical Example

#### Dataset
- $N = 100$ documents
- $n_t = 50$ (term appears in half the documents)

#### IDF Comparison
- **TF-IDF:**  
  $$
  \text{IDF} = \log\left(\frac{100}{50}\right) = \log(2) \approx 0.693
  $$

- **BM25:**  
  $$
  \text{IDF} = \log\left(\frac{100 - 50 + 0.5}{50 + 0.5} + 1\right) = \log\left(\frac{50.5}{50.5} + 1\right) = \log(2) \approx 0.693
  $$

Here, both are similar because $n_t$ is moderate.

#### For Rare Term ($n_t = 1$):
- **TF-IDF:**  
  $$
  \text{IDF} = \log\left(\frac{100}{1}\right) = \log(100) \approx 4.605
  $$

- **BM25:**  
  $$
  \text{IDF} = \log\left(\frac{100 - 1 + 0.5}{1 + 0.5} + 1\right) = \log\left(\frac{99.5}{1.5} + 1\right) = \log(67.33) \approx 4.208
  $$

BM25 gives a slightly lower score, smoothing the impact of very rare terms.

---

### Key Takeaways
- **TF-IDF IDF:** Simple but can overemphasize rare terms and lacks robustness.
- **BM25 IDF:** Smoothed and balanced, making it more suitable for real-world search and ranking tasks. 

BM25's IDF is tailored for document ranking, where the balance between common and rare terms is crucial for relevance.

## Intuition Behind $f(t, d)$ and $k_1$ in BM25

BM25 builds on the idea of term frequency ($f(t, d)$) and adds the parameter $k_1$ to fine-tune how term frequency impacts the relevance score. Here’s an intuitive breakdown:

---

### 1. $f(t, d)$: Term Frequency
- **What is it?**  
  $f(t, d)$ is the **raw count** of how many times a term $t$ appears in a document $d$. For example:
  - If the term "apple" appears 5 times in a document, $f(\text{"apple"}, d) = 5$.

- **Why does it matter?**  
  Higher term frequency often means the term is more important in the document. For example:
  - A document mentioning "apple" 20 times is likely more about apples than a document mentioning it once.

- **Limitations of Raw Term Frequency:**  
  If we only use $f(t, d)$, terms with very high frequencies could dominate the score, even if the extra appearances don’t add much value to the document's relevance.

---

### 2. $k_1$: Term Frequency Saturation
- **What is it?**  
  $k_1$ is a **tuning parameter** that controls how much weight we give to term frequency. It introduces a "saturation effect," meaning the score doesn't grow linearly with $f(t, d)$.

- **Why is it needed?**  
  Beyond a certain point, repeating a term doesn't necessarily make a document more relevant. For example:
  - A document mentioning "apple" 50 times is probably not 50 times more relevant than one mentioning it 10 times.

- **How it works:**  
  The term frequency $f(t, d)$ is scaled using $k_1$ in BM25:
  $$
  \text{TF factor} = \frac{f(t, d)}{f(t, d) + k_1}
  $$
  - When $f(t, d)$ is **low**, the score increases almost proportionally with $f(t, d)$.
  - When $f(t, d)$ is **high**, the denominator (with $k_1$) dampens the effect, preventing runaway scores.

---

### Impact of \( k_1 \):
- **Small $k_1$:**  
  - Makes BM25 behave more like a **binary match** — it cares less about term frequency.  
  - Example: Whether "apple" appears 2 times or 10 times, the score won't change much.
- **Large $k_1$:**  
  - Places **more emphasis** on term frequency but still with diminishing returns.  
  - Example: A document mentioning "apple" 20 times gets a higher score than one mentioning it 10 times, but the effect isn't linear.

---

### Intuition Through an Example

Imagine two documents about fruits:

#### Document 1:
- Mentions "apple" 3 times.

#### Document 2:
- Mentions "apple" 15 times.

#### With $k_1 = 1.5$:
- For Document 1:
  $$
  \text{TF factor} = \frac{3}{3 + 1.5} = \frac{3}{4.5} \approx 0.67
  $$
- For Document 2:
  $$
  \text{TF factor} = \frac{15}{15 + 1.5} = \frac{15}{16.5} \approx 0.91
  $$

Notice that the jump from 3 to 15 repetitions increases the score, but **not proportionally**.

#### With $k_1 = 0.5$:
- For Document 1:
  $$
  \text{TF factor} = \frac{3}{3 + 0.5} = \frac{3}{3.5} \approx 0.86
  $$
- For Document 2:
  $$
  \text{TF factor} = \frac{15}{15 + 0.5} = \frac{15}{15.5} \approx 0.97
  $$

With a smaller $k_1$, the effect of term frequency is further reduced, and the scores for the two documents become closer.

---

### Key Takeaways:
1. $f(t, d)$:  
   Measures how important a term is in a document based on its frequency.
   
2. $k_1$:  
   Smooths out the impact of $f(t, d)$, ensuring that term frequency doesn’t dominate the scoring process excessively.

3. **Together:**  
   BM25 balances relevance:
   - Accounts for frequency when a term appears a few times.
   - Dampens the effect when the term is repeated excessively.

## Code Implementation

In [9]:
from rank_bm25 import BM25Okapi

corpus = [
    "Apple Apple Banana",
    "Banana Mango Banana",
    "Cherry Cherry Cherry",
    "Grapes Grapes Berries Grapes",
    "Apple Banana Mango",
    "Blueberries Strawberries Apple",
    "Apple Banana Mango",
    "Grapes Grapes Grapes",
    "Blueberries Apple Strawberries",
    "Apple Banana Apple",
    "Cherry Cherry Mango Cherry",
    "Blueberries Strawberries Cherry",
]
tokenized_corpus = [doc.lower().split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

query = "banana mango"
tokenized_query = query.lower().split(" ")

doc_scores = bm25.get_scores(tokenized_query)
print(doc_scores)

docs = bm25.get_top_n(tokenized_query, tokenized_corpus, n=5)
docs

[0.3176789  1.10212021 0.         0.         0.96909597 0.
 0.96909597 0.         0.         0.3176789  0.56864878 0.        ]


[['banana', 'mango', 'banana'],
 ['apple', 'banana', 'mango'],
 ['apple', 'banana', 'mango'],
 ['cherry', 'cherry', 'mango', 'cherry'],
 ['apple', 'banana', 'apple']]

### BM25 from scratch

In [ ]:
import math
from collections import Counter


class BM25:
    def __init__(self, corpus: list[list[str]], k1: float = 1.2, b: float = 0.75):
        self.corpus = corpus
        self.k1 = k1
        self.b = b
        self.N = len(corpus)
        self.avg_dl = None

        self.nd, self.document_frequencies = self.initialize(self.corpus)
        self.idf = self.calculate_idf(self.nd)

    def initialize(self, corpus):
        nd = {}
        document_frequencies = []
        total_doc_len = 0
        for doc in corpus:
            total_doc_len += len(doc)
            document_frequencies.append(dict(Counter(doc)))
            for term in set(doc):
                if term not in nd:
                    nd[term] = 1
                else: 
                    nd[term] += 1

        self.avg_dl = total_doc_len / self.N
        return nd, document_frequencies

    def calculate_idf(self, nd):
        idf = {}
        for term in nd:
            idf[term] = math.log((self.N - nd[term] + 0.5) / (nd[term] + 0.5) + 1)

        return idf
    
    def get_scores(self, query: list[str]):
        for q in query:
            idf_q = self.idf[q]
            f_q = self.document_frequencies


bm25 = BM25(tokenized_corpus)
print(bm25.nd)
print(bm25.avg_dl)
print(bm25.document_frequencies)

{'banana': 5, 'apple': 6, 'mango': 4, 'cherry': 3, 'grapes': 2, 'berries': 1, 'blueberries': 3, 'strawberries': 3}
3.1666666666666665
[{'apple': 2, 'banana': 1}, {'banana': 2, 'mango': 1}, {'cherry': 3}, {'grapes': 3, 'berries': 1}, {'apple': 1, 'banana': 1, 'mango': 1}, {'blueberries': 1, 'strawberries': 1, 'apple': 1}, {'apple': 1, 'banana': 1, 'mango': 1}, {'grapes': 3}, {'blueberries': 1, 'apple': 1, 'strawberries': 1}, {'apple': 2, 'banana': 1}, {'cherry': 3, 'mango': 1}, {'blueberries': 1, 'strawberries': 1, 'cherry': 1}]
